<a href="https://colab.research.google.com/github/madoe001/torchrl-mario/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip3 install gym-super-mario-bros
!pip3 install nes_py
!pip3 install pyvirtualdisplay
!pip3 install torch torchvision torchaudio tensorboard moviepy tensordict torchrl
# !pip3 install git+https://github.com/Farama-Foundation/stable-retro.git

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
from typing import Optional
from tensordict import TensorDictBase
import retro
import PIL.Image
import torch
import itertools
import numpy as np
from collections import defaultdict
from tqdm import tqdm

from torchrl.envs import EnvBase
from torchrl.envs import (check_env_specs, TransformedEnv, Compose, Resize, ToTensorImage, GrayScale, CatFrames,
                          NoopResetEnv, StepCounter, DoubleToFloat, ObservationNorm, ExplorationType, RewardSum,
                          RewardScaling, FrameSkipTransform,
                          set_exploration_type)
from torchrl.envs.libs.gym import GymWrapper
from tensordict.nn import TensorDictModule
from tensordict.nn.distributions import OneHotCategorical
from torch import nn
from torchrl.modules.models import ConvNet, MLP
from torchrl.modules import ProbabilisticActor, ValueOperator, ActorValueOperator
from torchrl.data.replay_buffers import ReplayBuffer
from torchrl.data.replay_buffers.samplers import SamplerWithoutReplacement
from torchrl.data.replay_buffers.storages import LazyTensorStorage
from torchrl.objectives import ClipPPOLoss
from torchrl.objectives.value import GAE
from torchrl.record.loggers.tensorboard import TensorboardLogger
from torchrl.record import VideoRecorder
from tensordict import TensorDict
from torchrl.collectors import SyncDataCollector
from torchrl.data import (BoundedTensorSpec, CompositeSpec, UnboundedContinuousTensorSpec, BinaryDiscreteTensorSpec)

from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [5]:
from google.colab import drive
if 'google.colab' in str(get_ipython()):
  drive.mount('/content/drive')
  !python3 -m retro.import "/content/drive/MyDrive/ROMS/"
else:
  !python3 -m retro.import "."

/usr/local/lib/python3.10/dist-packages/pexpect/popen_spawn.py:60: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  self._read_thread.setDaemon(True)


Mounted at /content/drive
Importing SuperMarioBros-Nes
Importing SuperMarioBros-Nes
Imported 2 games


In [22]:
import gym
gym.__version__

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'0.25.2'

In [19]:
env.close()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
env = gym_super_mario_bros.make('SuperMarioBros-v3', new_step_api=True)
# env = JoypadSpace(env, SIMPLE_MOVEMENT)


In [35]:
env.reset()
done = True
truncate = True

for i in range(100):
    action = env.action_space.sample()
    _, reward, done, truncate, info = env.step(action)
    print(f"{action}: {reward}")
env.render()
# img = PIL.Image.fromarray(env.render())
# img
# for action in range(env.action_space.n):
#     for step in range(10):
#         if done | truncate:
#             state = env.reset()
#         _, reward, done, truncate, info = env.step(action)
#         print(reward)

171: 0.0
84: 0.0
251: -1.0
56: 0.0
248: 0.0
165: 0.0
66: 1.0
120: 0.0
137: 0.0
159: 0.0
35: 0.0
10: 0.0
141: 0.0
110: 0.0
242: 0.0
82: 0.0
182: 0.0
198: 0.0
170: 0.0
192: 1.0
53: 0.0
96: 0.0
222: -1.0
171: 0.0
42: 0.0
173: 0.0
84: 0.0
247: 0.0
168: 0.0
210: 0.0
253: 1.0
90: 0.0
67: 0.0
173: 0.0
197: 1.0
24: 0.0
254: 0.0
138: 1.0
243: 0.0
177: 1.0
245: 0.0
176: 1.0
104: 1.0
221: 0.0
252: 0.0
59: 0.0
130: 0.0
112: 0.0
65: 0.0
229: 0.0
26: 0.0
38: 0.0
48: 0.0
218: 0.0
137: 0.0
43: 0.0
177: 0.0
175: 0.0
104: 0.0
118: 0.0
242: 0.0
0: 0.0
132: 0.0
214: 0.0
95: 0.0
100: 0.0
222: 0.0
172: 0.0
163: 0.0
221: 0.0
166: 0.0
174: 0.0
255: 0.0
173: 0.0
84: 0.0
16: 0.0
118: 0.0
174: 0.0
166: 0.0
187: 0.0
162: 0.0
15: 0.0
174: 0.0
104: 0.0
71: 0.0
249: 0.0
198: 0.0
8: 0.0
40: 0.0
191: 0.0
97: 0.0
38: 0.0
218: 0.0
117: 0.0
224: 1.0
36: 1.0
17: -1.0
90: 1.0
49: 1.0
195: 0.0


NoSuchDisplayException: ignored

In [10]:
print(env.action_space)

Discrete(36)


In [11]:
for step in range(1000):
    if done | truncate:
        print("RESET FUNCTION")
        state = env.reset()
    action = env.action_space.sample()
    _, reward, done, truncate, info = env.step(action)
    # print(done, truncate, reward)
    # if step % 4 == 0:
    #    display(PIL.Image.fromarray(env.render()))
img = PIL.Image.fromarray(env.render())
# env.close()
img
env.reward_range

(-inf, inf)

In [ ]:
# Hyperparams
num_cells = 256  # @param
lr = 3e-4  # @param
max_grad_norm = 1.0  # @param

#FrameSkipping
frame_skip = 4  # @param
frames_per_batch = 1000 // frame_skip  # @param
# For a complete training, bring the number of frames up to 1M
total_frames = 1_000_000 // frame_skip  # @param

sub_batch_size = 64  # @param cardinality of the sub-samples gathered from the current data in the inner loop
num_epochs = 1  # @param optimisation steps per batch of data collected
clip_epsilon = (
    0.2  # @param clip value for PPO loss: see the equation in the intro for more context.
)
gamma = 0.99  # @param
lmbda = 0.95  # @param
entropy_eps = 1e-4  # @param

In [ ]:
device = "cuda" if torch.has_cuda else 'cpu'

print(f'Using device {device}')

In [ ]:
base_env = GymWrapper(env, device=device, frame_skip=4)

In [ ]:
print("observation_spec:", base_env.observation_spec)
print("state_spec:", base_env.state_spec)
print("reward_spec:", base_env.reward_spec)
logger = TensorboardLogger("TEST", log_dir="./logs")
recorder = VideoRecorder(logger, tag="SuperMario", )

env = TransformedEnv(
    base_env,
    Compose(
        # recorder,
        ActionResetEnv(noops=100,
                       action=6,
                       ),
        RewardScaling(0.0, 5.0),
        # NoopResetEnv(noops=1_000),
        FrameSkipTransform(frame_skip=4),
        ToTensorImage(),
        GrayScale(),
        Resize(84, 84),
        CatFrames(N=4, dim=-3),
        StepCounter(),
        # RewardSum(),
        ObservationNorm(standard_normal=True, in_keys=['pixels']),
        DoubleToFloat(in_keys=["pixels"]),
    )
)

In [ ]:
observation_norm_wrapper_index = 8
env.transform[observation_norm_wrapper_index].init_stats(num_iter=1000, reduce_dim=0, cat_dim=0)
print("normalization constant shape:", env.transform[observation_norm_wrapper_index].loc.shape)

In [ ]:
check_env_specs(env)

In [ ]:
num_outputs = env.action_spec.space.n
input_shape = env.observation_spec["pixels"].shape

In [ ]:
out_features = env.action_spec.shape[0]
print(out_features)

In [ ]:
common_cnn = ConvNet(
    activation_class=torch.nn.ReLU,
    num_cells=[32, 64, 64],
    kernel_sizes=[8, 4, 3],
    strides=[4, 2, 1],
)

common_cnn_output = common_cnn(torch.ones(input_shape))
common_mlp = MLP(
    in_features=common_cnn_output.shape[-1],
    activation_class=torch.nn.ReLU,
    activate_last_layer=True,
    out_features=512,
    num_cells=[],
)
common_mlp_output = common_mlp(common_cnn_output)

# Define shared net as TensorDictModule
common_module = TensorDictModule(
    module=torch.nn.Sequential(common_cnn, common_mlp),
    in_keys=["pixels"],
    out_keys=["common_features"],
)

# Define on head for the policy
policy_net = MLP(
    in_features=common_mlp_output.shape[-1],
    out_features=num_outputs,
    activation_class=torch.nn.ReLU,
    num_cells=[256],
)
policy_module = TensorDictModule(
    module=policy_net,
    in_keys=["common_features"],
    out_keys=["logits"],
)

# Add probabilistic sampling of the actions
policy_module = ProbabilisticActor(
    policy_module,
    in_keys=["logits"],
    spec=CompositeSpec(action=env.action_spec),
    # safe=True,
    distribution_class=OneHotCategorical,
    distribution_kwargs={},
    return_log_prob=True,
    default_interaction_type=ExplorationType.RANDOM,
)

# Define another head for the value
value_net = MLP(
    activation_class=torch.nn.ReLU,
    in_features=common_mlp_output.shape[-1],
    out_features=1,
    num_cells=[256],
)
value_module = ValueOperator(
    value_net,
    in_keys=["common_features"],
)

In [ ]:
actor_critic = ActorValueOperator(
    common_operator=common_module,
    value_operator=value_module,
    policy_operator=policy_module
).to(device)

In [ ]:
with torch.no_grad():
    td = env.rollout(max_steps=1000, break_when_any_done=False)
    td = actor_critic(td)

In [ ]:
collector = SyncDataCollector(
    env,
    actor_critic.get_policy_operator(),
    frames_per_batch=frames_per_batch,
    total_frames=total_frames,
    split_trajs=False,
    device=device,
)

In [ ]:
replay_buffer = ReplayBuffer(
    storage=LazyTensorStorage(frames_per_batch),
    sampler=SamplerWithoutReplacement(),
)

In [ ]:
advantage_module = GAE(
    gamma=gamma,
    lmbda=lmbda,
    value_network=actor_critic.get_value_operator(),
    average_gae=True
)

loss_module = ClipPPOLoss(
    actor=actor_critic.get_policy_operator(),
    critic=actor_critic.get_value_head(),
    clip_epsilon=clip_epsilon,
    entropy_bonus=bool(entropy_eps),
    entropy_coef=entropy_eps,
    # these keys match by default but we set this for completeness
    value_target_key=advantage_module.value_target_key,
    gamma=0.99,
)

optim = torch.optim.Adam(loss_module.parameters(), lr)

In [ ]:
logs = defaultdict(list)
pbar = tqdm(total=total_frames * frame_skip)
eval_str = ""

env.reset()

# We iterate over the collector until it reaches the total number of frames it was
# designed to collect:
for i, tensordict_data in enumerate(collector):
    # we now have a batch of data to work with. Let's learn something from it.
    for _ in range(num_epochs):
        # We'll need an "advantage" signal to make PPO work.
        # We re-compute it at each epoch as its value depends on the value
        # network which is updated in the inner loop.
        with torch.no_grad():
            advantage_module(tensordict_data)
        data_view = tensordict_data.reshape(-1)
        replay_buffer.extend(data_view.cpu())
        for _ in range(frames_per_batch // sub_batch_size):
            subdata = replay_buffer.sample(sub_batch_size)
            loss_vals = loss_module(subdata.to(device))
            loss_value = (
                    loss_vals["loss_objective"]
                    + loss_vals["loss_critic"]
                    + loss_vals["loss_entropy"]
            )

            # Optimization: backward, grad clipping and optim step
            loss_value.backward()
            # this is not strictly mandatory but it's good practice to keep
            # your gradient norm bounded
            torch.nn.utils.clip_grad_norm_(loss_module.parameters(), max_grad_norm)
            optim.step()
            optim.zero_grad()

    logs["reward"].append(tensordict_data["next", "reward"].mean().item())
    pbar.update(tensordict_data.numel() * frame_skip)
    cum_reward_str = (
        f"average reward={logs['reward'][-1]: 4.4f} (init={logs['reward'][0]: 4.4f})"
    )
    logs["step_count"].append(tensordict_data["step_count"].max().item())
    stepcount_str = f"step count (max): {logs['step_count'][-1]}"
    logs["lr"].append(optim.param_groups[0]["lr"])
    lr_str = f"lr policy: {logs['lr'][-1]: 4.4f}"

   #  recorder.dump(f"Episode_{i}")
    if i % 10 == 0:
        # We evaluate the policy once every 10 batches of data.
        # Evaluation is rather simple: execute the policy without exploration
        # (take the expected value of the action distribution) for a given
        # number of steps (1000, which is our env horizon).
        # The ``rollout`` method of the env can take a policy as argument:
        # it will then execute this policy at each step.
        with set_exploration_type(ExplorationType.MEAN), torch.no_grad():
            # execute a rollout with the trained policy
            eval_rollout = env.rollout(1000, actor_critic.get_policy_operator())
            logs["eval reward"].append(eval_rollout["next", "reward"].mean().item())
            logs["eval reward (sum)"].append(
                eval_rollout["next", "reward"].sum().item()
            )
            logs["eval step_count"].append(eval_rollout["step_count"].max().item())
            eval_str = (
                f"eval cumulative reward: {logs['eval reward (sum)'][-1]: 4.4f} "
                f"(init: {logs['eval reward (sum)'][0]: 4.4f}), "
                f"eval step-count: {logs['eval step_count'][-1]}"
            )
            del eval_rollout
    pbar.set_description(", ".join([eval_str, cum_reward_str, stepcount_str, lr_str]))

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
plt.subplot(2, 2, 1)
plt.plot(logs["reward"])
plt.title("training rewards (average)")
plt.subplot(2, 2, 2)
plt.plot(logs["step_count"])
plt.title("Max step count (training)")
plt.subplot(2, 2, 3)
plt.plot(logs["eval reward (sum)"])
plt.title("Return (test)")
plt.subplot(2, 2, 4)
plt.plot(logs["eval step_count"])
plt.title("Max step count (test)")
plt.show()